# Modulo

This problem asks to write a word surrounded by a pattern of characters.


## Tests

This function tests my program with the provided input and output files.

In [1]:
import glob
import subprocess
from timeit import default_timer as timer

def test():
    problem = 'okviri'
    start = timer()
    for infile in sorted(glob.iglob(f'{problem}/*.in.*')):
        outfile = infile.replace('in', 'out')
        command = f'python {problem}.py < {infile} | diff -w - {outfile}'
        print('Running on', infile)
        differences = subprocess.run(command, shell=True, text=True,
                                     stdout=subprocess.PIPE)
        if differences.stdout:
            print(differences.stdout)
    end = timer()
    print('Seconds:', end - start)

## Official solution

The approach is to set up a matrix of dots, replace some of them by the letters and the frame characetrs,
and finally print the matrix.

The matrix has 5 rows. Each frame takes 5 columns. 
Due to the overlap, the last column of a letter is the first of the next letter.
So in total there are 4 columns per letter plus a final column for the last letter.

We first write the Peter frames (with `#`) for every letter 
and then the Wendy frames (with `*`) for every third letter.

Letters are in the middle row (row 2), every 4 columns, starting at column 2 (the matrix is 0-indexed in Python).
We replace all dots that are at a Manhattan distance of 2 (see the previous problem) from the letter
by the frame character.

In [2]:
%%writefile okviri.py

def frame(column, letter, character):
    matrix[2][column] = letter
    for dr in range(-2, 3):
        for dc in range(-2, 3):
            if abs(dr) + abs(dc) == 2:
                matrix[2 + dr][column + dc] = character

word = input().rstrip()
length = len(word)

matrix = []
for row in range(5):
    matrix.append(['.'] * (4 * length + 1))

for index in range(length):
    frame(4 * index + 2, word[index], '#')
for index in range(2, length, 3):
    frame(4 * index + 2, word[index], '*')

for row in matrix:
    print(''.join(row))

Overwriting okviri.py


Calling `rstrip` shouldn't be necessary, as `input` removes the final newline, 
but for some reason the above test setup doesn't work without it.

In [3]:
test()

Running on okviri/okviri.in.1
Running on okviri/okviri.in.10
Running on okviri/okviri.in.2
Running on okviri/okviri.in.3
Running on okviri/okviri.in.4
Running on okviri/okviri.in.5
Running on okviri/okviri.in.6
Running on okviri/okviri.in.7
Running on okviri/okviri.in.8
Running on okviri/okviri.in.9
Seconds: 0.33079004300000014


### Variations

A more straightforward approach, that doesn't require coming up with index expressions, is as follows:

In [4]:
%%writefile okviri.py

def frame(column, letter, character):
    matrix[0][column] = character
    matrix[1][column-1] = matrix[1][column+1] = character
    matrix[2][column] = letter
    matrix[2][column-2] = matrix[2][column+2] = character
    matrix[3][column-1] = matrix[3][column+1] = character
    matrix[4][column] = character

word = input().rstrip()
length = len(word)

matrix = []
for row in range(5):
    matrix.append(['.'] * (4 * length + 1))

column = 2
for letter in word:
    frame(column, letter, '#')
    column += 4

column = 2
count = 1
for letter in word:
    if count % 3 == 0:
        frame(column, letter, '*')
    column += 4
    count += 1

for row in matrix:
    print(''.join(row))

Overwriting okviri.py


In [5]:
test()

Running on okviri/okviri.in.1
Running on okviri/okviri.in.10
Running on okviri/okviri.in.2
Running on okviri/okviri.in.3
Running on okviri/okviri.in.4
Running on okviri/okviri.in.5
Running on okviri/okviri.in.6
Running on okviri/okviri.in.7
Running on okviri/okviri.in.8
Running on okviri/okviri.in.9
Seconds: 0.34144987400000004


## Printing directly

It's also possible to print line by line, 4 characters at a time for each letter.
In the middle row, a frame starts with `*` if this or the previous frame is a Wendy frame.
We need a Boolean to remember the kind of frame.

In [6]:
%%writefile okviri.py

word = input().rstrip()
length = len(word)

def first_and_last():
    for index in range(length):
        if (index + 1) % 3 == 0:
            print('..*.', end='')
        else:
            print('..#.', end='')
    print('.')

def second_and_penultimate():
    for index in range(length):
        if (index + 1) % 3 == 0:
            print('.*.*', end='')
        else:
            print('.#.#', end='')
    print('.')

first_and_last()
second_and_penultimate()

previous_wendy = False
for index, letter in enumerate(word):
    this_wendy = (index + 1) % 3 == 0
    if this_wendy or previous_wendy:
        print(f'*.{letter}.', end='')
    else:
        print(f'#.{letter}.', end='')
    previous_wendy = this_wendy
print('*' if previous_wendy else '#')

second_and_penultimate()
first_and_last()

Overwriting okviri.py


In [7]:
test()

Running on okviri/okviri.in.1
Running on okviri/okviri.in.10
Running on okviri/okviri.in.2
Running on okviri/okviri.in.3
Running on okviri/okviri.in.4
Running on okviri/okviri.in.5
Running on okviri/okviri.in.6
Running on okviri/okviri.in.7
Running on okviri/okviri.in.8
Running on okviri/okviri.in.9
Seconds: 0.33062503499999996


## Concluding remarks

I was expecting the last solution to be the slowest, due to its repetitive small I/O operations, 
but it turns out that they all take about the same time. 
Maybe with longer words the differences become noticeable. 